# Kernel Optimizations

In this notebook, we will explore some kernel optimizations. More specifically, we will take a look at:

1. using a wide data bus to read more elements in parallel
2. using multiple memory banks
3. overlapping communication and computation

To this purpose, we will use the `vadd_wide` and `vadd_wide_multibanks` kernels available in the kernel optimization overlay. The two kernels similarly implement a vector addition using 512 bit datapaths, therefore accessing 16 elements at a time (512/32, since each element is 32 bit). The only difference is that `vadd_wide_multibanks` uses a different memory bank for each buffer involved, while `vadd_wide` does not.

![vadd-wide](img/vadd-wide.png "Vector Addition With Wide Data Busses")

## Example 1: using wide data bus

As usual, the first thing we do is download the overlay on the FPGA. Then, we assign the wide vector addition kernel to a variable called `vadd_wide`. 

Since the kernel optimization overlay uses multiple memory banks, we will need to explicitly set the target memory bank when allocating buffers, or `pynq.allocate` will raise an exception.

When the overlay uses a single memory bank, that memory bank is picked automatically as default allocation target. However, when multiple banks are used, there is no way of guessing a default target.

Therefore, we need to know to which memory bank each buffer is associated. We can do this by using the `.args` property, that will print all the arguments needed by the kernel, their type and, when they are buffers, the associated memory bank.

In [1]:
import pynq
ol = pynq.Overlay("kernel_opt.xclbin")

vadd_wide = ol.vadd_wide_1
vadd_wide.args

{'a': XrtArgument(name='a', index=1, type='ap_uint<512>*', mem='bank1'),
 'b': XrtArgument(name='b', index=2, type='ap_uint<512>*', mem='bank1'),
 'c': XrtArgument(name='c', index=3, type='ap_uint<512>*', mem='bank1'),
 'count': XrtArgument(name='count', index=4, type='int', mem=None)}

The three buffers `a`, `b` and `c` in this case are associated to bank1, so we will need to specify `target=ol.bank1` when using `pynq.allocate`. For this kernel, `a` and `b` represent the input buffers, while `c` represents the output. We therefore instantiate 3 buffers, `in1` and `in2` for the input, and `out` for the output.

We rely on numpy to initialize the two input buffers with random integers in the range [0, 1000).

In [2]:
import numpy as np
size = 1024*1024

in1 = pynq.allocate((1024, 1024), 'u4', target=ol.bank1)
in2 = pynq.allocate((1024, 1024), 'u4', target=ol.bank1)
out = pynq.allocate((1024, 1024), 'u4', target=ol.bank1)

in1[:] = np.random.randint(1000, size=(1024, 1024), dtype='u4')
in2[:] = np.random.randint(1000, size=(1024, 1024), dtype='u4')

We then run the kernel, first syncing the two input buffers to update the FPGA memory, and then, after kernel completion, sync the output buffer to transfer data back to the host memory. 

We use the `%%timeit` magic to get the average execution time.

In [3]:
%%timeit
in1.sync_to_device()
in2.sync_to_device()
vadd_wide.call(in1, in2, out, size)
out.sync_from_device()

3.17 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As you can notice, the execution time is much lower with respect the one measured in the [introduction](../1-introduction/introduction.ipynb) notebook for the vector addition kernel used there. This is because the `vadd_wide` kernel is using a much wider data bus, reading (and writing) 16 elements at a time for each buffer, instead of one.

Moreover, in the previous cell most of the time is actually spent to syncing. Indeed, if we just measure the average run time for the sole kernel `.call()`

In [4]:
%%timeit
vadd_wide.call(in1, in2, out, size)

748 µs ± 682 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We can see that this accounts for just a fraction of the total time measured earlier. This will be useful later when we move to the second example.

Let's end by performing the usual comparison with software using `numpy.array_equal`

In [5]:
np.array_equal(out, in1 + in2)

True

And delete the used buffers

In [ ]:
del in1
del in2
del out

## Example 2: using multiple memory banks

We now shift our focus to the wide vector addition kernel that uses multiple memory banks. As we did previously, we assign the kernel IP to a variable, in this case `vadd_wide_mb` and we print out the arguments using `.args`, to know which memory banks are used by each buffer involved.

In [6]:
vadd_wide_mb = ol.vadd_wide_multibanks_1
vadd_wide_mb.args

{'a': XrtArgument(name='a', index=1, type='ap_uint<512>*', mem='bank0'),
 'b': XrtArgument(name='b', index=2, type='ap_uint<512>*', mem='bank1'),
 'c': XrtArgument(name='c', index=3, type='ap_uint<512>*', mem='bank2'),
 'count': XrtArgument(name='count', index=4, type='int', mem=None)}

You can see that in this case `a` uses `bank0`, `b` uses `bank1`, and `c` uses `bank2`, so each buffer uses a different memory bank. We use `pynq.allocate` again, this time with respectively `bank0`, `bank1` and `bank2` as targets.

We then use numpy one more time to initialize the input buffers with random integers, as we did before.

In [7]:
in1 = pynq.allocate((1024, 1024), 'u4', target=ol.bank0)
in2 = pynq.allocate((1024, 1024), 'u4', target=ol.bank1)
out = pynq.allocate((1024, 1024), 'u4', target=ol.bank2)

in1[:] = np.random.randint(1000, size=(1024, 1024), dtype='u4')
in2[:] = np.random.randint(1000, size=(1024, 1024), dtype='u4')

We can now execute the kernel and measure the execution time with `%%timeit`. Since we are using different memory banks for each buffer, there is no contention on accessing data and therefore we expect this version of the wide vector addition to be faster.

In [8]:
%%timeit
in1.sync_to_device()
in2.sync_to_device()
vadd_wide_mb.call(in1, in2, out, size)
out.sync_from_device()

2.52 ms ± 21.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Also in this case, most of the time is spent transferring data between the host and the FPGA memory. So, as we did before, let's just measure the execution time for `.call` only to truly appreciate the speedup we obtained by using multiple banks

In [9]:
%%timeit
vadd_wide_mb.call(in1, in2, out, size)

299 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


And to conclude this example, we perform the usual comparison with software

In [10]:
np.array_equal(out, in1 + in2)

True

And delete the used buffers

In [ ]:
del in1
del in2
del out

## Example 3: overlapping computation and communication

In this last example, we will see how to call a kernel asynchronously and exploit this feature to overlap communication and computation, resulting in a performance increase.

For this example, we are still going to use the `vadd_wide_mb` kernel, but we will call it iteratively in a loop, and let it operate on slices of the buffers involved.

We allocate new buffers this time little bigger (10x1024x1024 instead of just 1024x1024). This is not strictly needed, but it is going to make things a little more clear. As always, we rely on numpy to initialize the input buffers with random integers.

In [11]:
in1_iter = pynq.allocate((10, 1024, 1024), 'u4', target=ol.bank0)
in2_iter = pynq.allocate((10, 1024, 1024), 'u4', target=ol.bank1)
out_iter = pynq.allocate((10, 1024, 1024), 'u4', target=ol.bank2)

in1_iter[:] = np.random.randint(1000, size=(10, 1024, 1024), dtype='u4')
in2_iter[:] = np.random.randint(1000, size=(10, 1024, 1024), dtype='u4')

### Serial execution

We then execute the kernel in a for loop, letting it work on slices of the buffers obtained by iterating on the outer dimension. For this first execution, we are not performing any overlap. 

We measure the average execution time using `%%timeit` to establish a baseline.

In [12]:
%%timeit
for i in range(10):
    in1_iter[i].sync_to_device()
    in2_iter[i].sync_to_device()
    
    vadd_wide_mb.call(in1_iter[i], in2_iter[i], out_iter[i], size)
    
    out_iter[i].sync_from_device()

26 ms ± 99.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


And we then compare, as usual, the results with software, using numpy.

In [13]:
np.array_equal(out_iter, in1_iter + in2_iter)

True

### Overlapped execution

Let's now explore how to speedup the execution by overlapping communicaiton and computation.

![overlap](img/overlap.png "Overlap Computation and Communication")

Although this is not strictly needed, let's initialize again the input buffers with new random values, and set the output buffer to all zeroes before continuing. This way we can be 100% sure that the computation is correct when we perform the comparison with software

In [14]:
in1_iter[:] = np.random.randint(1000, size=(10, 1024, 1024), dtype='u4')
in2_iter[:] = np.random.randint(1000, size=(10, 1024, 1024), dtype='u4')
out_iter[:] = 0

We now execute again the kernel iteratively, but this time, we rely on [`.start()`](https://pynq.readthedocs.io/en/v2.5/pynq_package/pynq.overlay.html#pynq.overlay.DefaultIP.start) instead of `.call()` to do it asynchronously.
The `.start()` function returns a handle that we can then use to `.wait()` for the kernel to complete. In this way, the host does not need to wait for the kernel to finish after it has been started, unless you explicitly do it using `.wait()`.

To achieve the communication and computation overlap, after starting the kernel we transfer the host buffer to the FPGA, for each iteration of the loop, the input buffers' slices at index `i+1` (i.e. the data for the next iteration), and we transfer back the output buffer's slice at index `i-1` (i.e. the previous iteration), before waiting for the kernel to finish the current iteration `i`. To achieve this overlap safely, we have to add some safeguards (the `if` statments that check for the current index when syncing), and also preload the input buffers at index 0 before the loop starts, and copy back the output buffer at index 9 after the loop has ended.

In [15]:
%%timeit
in1_iter[0].sync_to_device()
in2_iter[0].sync_to_device()
for i in range(10):
    handle = vadd_wide_mb.start(in1_iter[i], in2_iter[i], out_iter[i], size)
    
    if i < 9:
        in1_iter[i+1].sync_to_device()
        in2_iter[i+1].sync_to_device()
    
    if i > 0:
        out_iter[i-1].sync_from_device()

    handle.wait()

out_iter[9].sync_from_device()

23 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


By measuring the average execution time, we can see that this optimization allowed us in this case an approximately 13% performance gain.

Finally, we compare the result with software, proving that the computation on the FPGA was indeed carried out correctly.

In [16]:
np.array_equal(out_iter, in1_iter + in2_iter)

True

## Cleaning up

We end this notebook by freeing up the buffers still allocated, and the overlay context, so that the FPGA can be reused with a different application.

In [17]:
del in1_iter
del in2_iter
del out_iter
ol.free()

Copyright (C) 2020 Xilinx, Inc